In [254]:
#REQUIREMENTS
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import datetime
import time
from api_keys import g_key
#from api_keys import weather_api_key


#Setting display options
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)

In [170]:
#Reading Cleaned Data Frame
test_points = pd.read_csv("movilidad.csv",index_col=False)


In [171]:
#analyzing DF
test_points

,Unnamed: 0,id_via,id_soc,clasif_viaje,lugar_origen,municipio_origen,estado_origen,colonia_origen,tipo_lugar_destino_,municipio_destino,estado_destino,colonia_destino,proposito_viaje,hora_comienzo,minuto_comienzo,hora_termino,minuto_termino,¿Mexicable?,¿Auto?,# Auto,¿Colectivo/Micro?,# Colectivo/Micro,¿Taxi (App)?,# Taxi (App),¿Taxi?,#Taxi,¿Metro?,# Metro,¿Autobus RTP?,# Autobus RTP,¿Bicicleta?,#Veces Bicicleta,¿Autobus?,# Autobus,¿Moto?,# Moto,¿Trolebus?,# Trolebus,¿Metrobus?,# Metrobus,¿Tren ligero?,# Tren ligero,¿Tren suburbano?,# Tren suburbano,¿Camino?,# Camino,#Mexicable,¿Bicitaxi?,# Bicitaxi,¿Mototaxi?,#Mototaxi,¿Transporte escolar?,#Transporte escolar,¿Transporte de personal?,#Transporte de personal,¿Otro medio?,# Otro medio,¿Condujo?,estrato,factor,upm_dis,est_dis,tloc,sexo,edad
0,0,2936,1268,Viaje realizado entre semana (Sección V),Su hogar,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,Oficina,Miguel Hidalgo,Ciudad de México,Chapultepec-Polanco,Ir al trabajo,8,0,9,0,2,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,NaN,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,26
1,1,2937,1268,Viaje realizado entre semana (Sección V),Oficina,Miguel Hidalgo,Ciudad de México,Chapultepec-Polanco,Su hogar,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,Ir al hogar,14,0,15,30,2,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,NaN,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,26
2,2,2938,1268,Viaje realizado en sábado (Sección VI),Su hogar,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,Otra vivienda,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,"Convivir a(amigos o familiares), deportes o re...",17,0,17,5,2,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,NaN,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,26
3,3,2939,1268,Viaje realizado en sábado (Sección VI),Otra vivienda,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,Su hogar,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,Ir al hogar,23,0,23,5,2,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,NaN,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,26
4,4,2940,1269,Viaje realizado entre semana (Sección V),Su hogar,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,Oficina,Miguel Hidalgo,Ciudad de México,Chapultepec-Polanco,Ir al trabajo,8,0,9,0,2,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,NaN,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531589,531589,530057,199596,Viaje realizado entre semana (Sección V),Oficina,Tlalpan,Ciudad de México,Villa Olim

In [5]:
del test_points["Unnamed: 0"]

In [7]:
#Testing Data Frame Structure
orig = test_points.groupby(['lugar_origen','colonia_origen']).count()['id_soc']
orig_df = pd.DataFrame({"Number of users":orig})

In [9]:
#Focusing on routes initiated @ home
home_df = test_points.loc[test_points['lugar_origen'] == "Su hogar"]

home_df

,id_via,id_soc,clasif_viaje,lugar_origen,municipio_origen,estado_origen,colonia_origen,tipo_lugar_destino_,municipio_destino,estado_destino,...,¿Otro medio?,# Otro medio,¿Condujo?,estrato,factor,upm_dis,est_dis,tloc,sexo,edad
0,2936,1268,Viaje realizado entre semana (Sección V),Su hogar,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,Oficina,Miguel Hidalgo,Ciudad de México,...,No utilizó,NaN,NaN,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,26
2,2938,1268,Viaje realizado en sábado (Sección VI),Su hogar,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,Otra vivienda,Cuauhtémoc,Ciudad de México,...,No utilizó,NaN,NaN,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,26
4,2940,1269,Viaje realizado entre semana (Sección V),Su hogar,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,Oficina,Miguel Hidalgo,Ciudad de México,...,No utilizó,NaN,NaN,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,22
6,2942,1269,Viaje realizado en sábado (Sección VI),Su hogar,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,"Restaurante, bar, cafetería",Cuauhtémoc,Ciudad de México,...,No utilizó,NaN,NaN,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,22
8,2944,1270,Viaje realizado entre semana (Sección V),Su hogar,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,Escuela,Cuauhtémoc,Ciudad de México,...,No utilizó,NaN,NaN,Medio alto,236,88,40,"Localidades mayores de 100,000 habitantes",Hombre,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531584,530052,199594,Viaje realizado en sábado (Sección VI),Su hogar,Tizayuca,Hidalgo,Tizayuca,Oficina,Tizayuca,Hidalgo,...,No utilizó,NaN,NaN,Medio bajo,68,12313,5460,"Localidades de 15,000 a 99,999 habitantes",Hombre,54
531586,530054,199595,Viaje realizado entre semana (Sección V),Su hogar,Tizayuca,Hidalgo,Tizayuca,Centro cultural o área recreativa,Tizayuca,Hidalgo,...,No utilizó,NaN,NaN,Medio bajo,68,12313,5460,"Localidades de 15,000 a 99,999 habitantes",Mujer,57
531588,530056,199596,Viaje realizado entre semana (Sección V),Su hogar,Tizayuca,Hidalgo,Tizayuca,Oficina,Tlalpan,Ciudad de México,...,No utilizó,NaN,NaN,Medio bajo,68,12313,5460,"Localidades de 15,000 a 99,999 habitantes",Hombre,26
531590,530058,199597,Viaje realizado en sábado (Sección VI),Su hogar,Tizayuca,Hidalgo,Tizayuca,Fábrica o taller,Tecámac,México,...,No utilizó,NaN,NaN,Medio bajo,68,12313,5460,"Localidades de 15,000 a 99,999 habitantes",Hombre,51


In [11]:
#Selecting routes which it's endpoint is "work"
tra_ida_df = home_df.loc[home_df['proposito_viaje'] == 'Ir al trabajo']
#Selecting routes that occur within weekdays
tra_ida_df = tra_ida_df.loc[tra_ida_df['clasif_viaje'] == 'Viaje realizado entre semana (Sección V)']


In [176]:
#Focusing on trips that occur within CDMX and the State of Mexico, focusing on what is called the Metropolitan Area
#Excluding one municipality from the state of Hidalgo that is considered as part of "Zona Conurbada" 
#that would increase significantly route extraction from google maps
#Extracting routes to-from state of Mexico - CDMX
dest_list = ['Ciudad de México', 'México']
cdmx_df_o = tra_ida_df.loc[(tra_ida_df['estado_origen'].isin(dest_list)) | (tra_ida_df['estado_destino'].isin(dest_list))]
#edomex_df_o = tra_ida_df.loc[tra_ida_df['estado_origen'] == 'México']
#cdmx_df_d = tra_ida_df.loc[tra_ida_df['estado_destino'] == 'Ciudad de México']
#edomex_df_d = tra_ida_df.loc[tra_ida_df['estado_destino'] == 'México']
cdmx_df_o.loc[cdmx_df_o.duplicated(subset = ['id_via']),"id_via"]

Series([], Name: id_via, dtype: int64)

In [13]:
#Integrating Dataframes and dropping any possible duplicate
frames = [edomex_df_o, edomex_df_d, cdmx_df_o, cdmx_df_d]
result = pd.concat(frames)
result.drop_duplicates()
result = result.drop_duplicates()

In [177]:
#exclude_list = ["Amecameca", "Apaxco"]
#filter_df = cdmx_df_o.loc[~cdmx_df_o["municipio_origen"].isin(exclude_list)]
#filter_df

,id_via,id_soc,clasif_viaje,lugar_origen,municipio_origen,estado_origen,colonia_origen,tipo_lugar_destino_,municipio_destino,estado_destino,colonia_destino,proposito_viaje,hora_comienzo,minuto_comienzo,hora_termino,minuto_termino,¿Mexicable?,¿Auto?,# Auto,¿Colectivo/Micro?,# Colectivo/Micro,¿Taxi (App)?,# Taxi (App),¿Taxi?,#Taxi,¿Metro?,# Metro,¿Autobus RTP?,# Autobus RTP,¿Bicicleta?,#Veces Bicicleta,¿Autobus?,# Autobus,¿Moto?,# Moto,¿Trolebus?,# Trolebus,¿Metrobus?,# Metrobus,¿Tren ligero?,# Tren ligero,¿Tren suburbano?,# Tren suburbano,¿Camino?,# Camino,#Mexicable,¿Bicitaxi?,# Bicitaxi,¿Mototaxi?,#Mototaxi,¿Transporte escolar?,#Transporte escolar,¿Transporte de personal?,#Transporte de personal,¿Otro medio?,# Otro medio,¿Condujo?,estrato,factor,upm_dis,est_dis,tloc,sexo,edad
0,2936,1268,Viaje realizado entre semana (Sección V),Su hogar,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,Oficina,Miguel Hidalgo,Ciudad de México,Chapultepec-Polanco,Ir al trabajo,8,0,9,0,2,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,NaN,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,26
4,2940,1269,Viaje realizado entre semana (Sección V),Su hogar,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,Oficina,Miguel Hidalgo,Ciudad de México,Chapultepec-Polanco,Ir al trabajo,8,0,9,0,2,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,NaN,Medio alto,244,87,40,"Localidades mayores de 100,000 habitantes",Mujer,22
22,2958,1273,Viaje realizado entre semana (Sección V),Su hogar,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,Otra vivienda,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,Ir al trabajo,7,30,7,35,2,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,NaN,Medio alto,236,88,40,"Localidades mayores de 100,000 habitantes",Mujer,56
30,2966,1274,Viaje realizado entre semana (Sección V),Su hogar,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,"Comercio, mercado, tienda o centro comercial",Cuauhtémoc,Ciudad de México,Centro Histórico,Ir al trabajo,6,0,7,15,2,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,NaN,Medio alto,236,88,40,"Localidades mayores de 100,000 habitantes",Mujer,33
42,2978,1278,Viaje realizado entre semana (Sección V),Su hogar,Cuauhtémoc,Ciudad de México,Buenavista-Reforma,"Comercio, mercado, tienda o centro comercial",Cuauhtémoc,Ciudad de México,Buenavista-Reforma,Ir al trabajo,7,20,7,40,2,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,NaN,Medio alto,236,88,40,"Localidades mayores de 100,000 habitantes",Hombre,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531202,529670,199474,Viaje realizado entre semana (Sección V),Su hogar,Tizayuca,Hidalgo,Tizayuca,Cen

In [282]:
#Dropping municipalities that are not part of the defined "Zona conurbada del valle de México"
#https://en.wikipedia.org/wiki/Greater_Mexico_City


indexNames = result[result['municipio_origen'] == "Amecameca"].index
result.drop(indexNames , inplace=True)
indexNames = result[result['municipio_origen'] == "Apaxco"].index
result.drop(indexNames , inplace=True)
indexNames = result[result['municipio_origen'] == "Atlautla"].index
result.drop(indexNames , inplace=True)
indexNames = result[result['municipio_origen'] == "Axapusco"].index
result.drop(indexNames , inplace=True)
indexNames = result[result['municipio_origen'] == "Ayapango"].index
result.drop(indexNames , inplace=True)
indexNames = result[result['municipio_origen'] == "Ecatzingo"].index
result.drop(indexNames , inplace=True)
indexNames = result[result['municipio_origen'] == "Hueypoxtla"].index
result.drop(indexNames , inplace=True)
indexNames = result[result['municipio_origen'] == "Jilotzingo"].index
result.drop(indexNames , inplace=True)
indexNames = result[result['municipio_origen'] == "Juchipetec"].index
result.drop(indexNames , inplace=True)
indexNames = result[result['municipio_origen'] == "Nopaltepec"].index
result.drop(indexNames , inplace=True)
indexNames = result[result['municipio_origen'] == "Otumba"].index
result.drop(indexNames , inplace=True)
indexNames = result[result['municipio_origen'] == "Ozumba"].index
result.drop(indexNames , inplace=True)
indexNames = result[result['municipio_origen'] == "San Martín de las Pirámides"].index
result.drop(indexNames , inplace=True)
indexNames = result[result['municipio_origen'] == "Temascalapa"].index
result.drop(indexNames , inplace=True)
indexNames = result[result['municipio_origen'] == "Tenango del Aire"].index
result.drop(indexNames , inplace=True)
indexNames = result[result['municipio_origen'] == "Tepetlixpa"].index
result.drop(indexNames , inplace=True)
indexNames = result[result['municipio_origen'] == "Tequixquiac"].index
result.drop(indexNames , inplace=True)
indexNames = result[result['municipio_origen'] == "Tizayuca"].index
result.drop(indexNames , inplace=True)
indexNames = result[result['municipio_origen'] == "Tonanitla"].index
result.drop(indexNames , inplace=True)
indexNames = result[result['municipio_origen'] == "Villa del Carbón"].index
result.drop(indexNames , inplace=True)

In [283]:
result

,id_via,id_soc,clasif_viaje,lugar_origen,municipio_origen,estado_origen,colonia_origen,tipo_lugar_destino_,municipio_destino,estado_destino,colonia_destino,proposito_viaje,hora_comienzo,minuto_comienzo,hora_termino,minuto_termino,¿Mexicable?,¿Auto?,# Auto,¿Colectivo/Micro?,# Colectivo/Micro,¿Taxi (App)?,# Taxi (App),¿Taxi?,#Taxi,¿Metro?,# Metro,¿Autobus RTP?,# Autobus RTP,¿Bicicleta?,#Veces Bicicleta,¿Autobus?,# Autobus,¿Moto?,# Moto,¿Trolebus?,# Trolebus,¿Metrobus?,# Metrobus,¿Tren ligero?,# Tren ligero,¿Tren suburbano?,# Tren suburbano,¿Camino?,# Camino,#Mexicable,¿Bicitaxi?,# Bicitaxi,¿Mototaxi?,#Mototaxi,¿Transporte escolar?,#Transporte escolar,¿Transporte de personal?,#Transporte de personal,¿Otro medio?,# Otro medio,¿Condujo?,estrato,factor,upm_dis,est_dis,tloc,sexo,edad
238546,242828,83174,Viaje realizado entre semana (Sección V),Su hogar,Huixquilucan,México,Localidades Pte Chamapa-Lechería,"Comercio, mercado, tienda o centro comercial",Huixquilucan,México,Localidades Pte Chamapa-Lechería,Ir al trabajo,8,0,8,30,2,Utilizó,1.0,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No,Medio bajo,46,5377,2330,"Localidades de 2,500 a 14,999 habitantes",Mujer,66
238550,242832,83178,Viaje realizado entre semana (Sección V),Su hogar,Huixquilucan,México,Localidades Pte Chamapa-Lechería,Fábrica o taller,Huixquilucan,México,Localidades Pte Chamapa-Lechería,Ir al trabajo,10,0,10,15,2,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,NaN,Medio bajo,46,5377,2330,"Localidades de 2,500 a 14,999 habitantes",Mujer,40
238562,242844,83182,Viaje realizado entre semana (Sección V),Su hogar,Huixquilucan,México,Localidades Pte Chamapa-Lechería,Escuela,Huixquilucan,México,Localidades Pte Chamapa-Lechería,Ir al trabajo,7,20,7,30,2,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,NaN,Medio bajo,46,5377,2330,"Localidades de 2,500 a 14,999 habitantes",Mujer,46
238564,242846,83183,Viaje realizado entre semana (Sección V),Su hogar,Huixquilucan,México,Localidades Pte Chamapa-Lechería,Escuela,Cuautitlán Izcalli,México,Centro Urbano-CC Cuautitlán Izcalli,Ir al trabajo,5,0,7,0,2,No utilizó,NaN,Utilizó,1.0,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,NaN,Medio bajo,46,5377,2330,"Localidades de 2,500 a 14,999 habitantes",Mujer,42
238566,242848,83186,Viaje realizado entre semana (Sección V),Su hogar,Huixquilucan,México,Localidades Pte Chamapa-Lechería,Oficina,Gustavo A. Madero,Ciudad de México,Distrito no especificado dentro de la ZMVM,Ir al trabajo,7,0,8,30,2,Utilizó,1.0,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Sí,Medio bajo,46,5377,2330,"Localidades de 2,500 a 14,999 habitantes",Hombre,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [19]:
#Testing structure of the data frame, trying to find best way to summarize most important routes
or_list =result.groupby(['municipio_origen']).nunique()
orig = result.groupby(['municipio_origen', 'colonia_origen']).count()['id_soc']
orig_col_or = result.groupby(['colonia_origen']).count()['id_soc']


In [21]:
#Insight, there is a large number of intra-municipality and intra-neighbourhood trips, dropping
results_new = result.loc[(result.colonia_origen != result.colonia_destino), :]
results_new = result.loc[(result.municipio_origen != result.municipio_destino), :]


In [45]:
#pd.set_option('display.max_rows', 500)

pd.options.display.max_rows = 100
pd.set_option("display.max_rows", 500)

results_new

,id_via,id_soc,clasif_viaje,lugar_origen,municipio_origen,estado_origen,colonia_origen,tipo_lugar_destino_,municipio_destino,estado_destino,colonia_destino,proposito_viaje,hora_comienzo,minuto_comienzo,hora_termino,minuto_termino,¿Mexicable?,¿Auto?,# Auto,¿Colectivo/Micro?,# Colectivo/Micro,¿Taxi (App)?,# Taxi (App),¿Taxi?,#Taxi,¿Metro?,# Metro,¿Autobus RTP?,# Autobus RTP,¿Bicicleta?,#Veces Bicicleta,¿Autobus?,# Autobus,¿Moto?,# Moto,¿Trolebus?,# Trolebus,¿Metrobus?,# Metrobus,¿Tren ligero?,# Tren ligero,¿Tren suburbano?,# Tren suburbano,¿Camino?,# Camino,#Mexicable,¿Bicitaxi?,# Bicitaxi,¿Mototaxi?,#Mototaxi,¿Transporte escolar?,#Transporte escolar,¿Transporte de personal?,#Transporte de personal,¿Otro medio?,# Otro medio,¿Condujo?,estrato,factor,upm_dis,est_dis,tloc,sexo,edad
238564,242846,83183,Viaje realizado entre semana (Sección V),Su hogar,Huixquilucan,México,Localidades Pte Chamapa-Lechería,Escuela,Cuautitlán Izcalli,México,Centro Urbano-CC Cuautitlán Izcalli,Ir al trabajo,5,0,7,0,2,No utilizó,NaN,Utilizó,1.0,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,NaN,Medio bajo,46,5377,2330,"Localidades de 2,500 a 14,999 habitantes",Mujer,42
238576,242858,83192,Viaje realizado entre semana (Sección V),Su hogar,Huixquilucan,México,Localidades Pte Chamapa-Lechería,Oficina,Miguel Hidalgo,Ciudad de México,Chapultepec-Polanco,Ir al trabajo,12,30,14,0,2,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,NaN,Medio bajo,46,5377,2330,"Localidades de 2,500 a 14,999 habitantes",Hombre,27
238634,242916,83225,Viaje realizado entre semana (Sección V),Su hogar,Huixquilucan,México,Localidades Pte Chamapa-Lechería,Otro,Naucalpan de Juárez,México,CC Interlomas-Lomas de Tecamachalco,Ir al trabajo,5,10,6,10,2,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,NaN,Medio bajo,43,5379,2330,"Localidades de 2,500 a 14,999 habitantes",Hombre,36
238670,242952,83235,Viaje realizado entre semana (Sección V),Su hogar,Huixquilucan,México,Localidades Pte Chamapa-Lechería,"Comercio, mercado, tienda o centro comercial",Cuajimalpa de Morelos,Ciudad de México,Cuajimalpa,Ir al trabajo,10,0,11,0,2,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,NaN,Medio bajo,43,5379,2330,"Localidades de 2,500 a 14,999 habitantes",Hombre,19
238672,242954,83237,Viaje realizado entre semana (Sección V),Su hogar,Huixquilucan,México,Localidades Pte Chamapa-Lechería,Otro,Miguel Hidalgo,Ciudad de México,Chapultepec-Polanco,Ir al trabajo,7,0,8,8,2,No utilizó,NaN,Utilizó,1.0,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,Utilizó,1.0,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,No utilizó,NaN,NaN,Medio bajo,43,5379,2330,"Localidades de 2,500 a 14,999 habitantes",Hombre,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
530883,531471,200085,Viaje realizado en

In [23]:
#There are a large number of trips that end outside the metropolitan zone, and some that the user didn't answer
indexNames = results_new[results_new['colonia_destino'] == "Distrito no especificado dentro de la ZMVM"].index
results_new.drop(indexNames , inplace=True)
indexNames = results_new[results_new['colonia_destino'] == "Fuera de la ZMVM"].index
results_new.drop(indexNames , inplace=True)

/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [182]:
#Testing the structure of the dataframe via pivots
#Objective is to find which are the most prominent routes between municipalities and neighbourhoods
muni_table = pd.pivot_table(results_new, values='id_via', index=['municipio_origen', 'municipio_destino'], aggfunc=pd.Series.nunique, dropna = True)



In [183]:
muni_table

id_via
municipio_origen municipio_destino           
Acolman          Atenco                     6
                 Atizapán de Zaragoza       2
                 Axapusco                   1
                 Azcapotzalco               7
                 Benito Juárez              6
...                                       ...
Álvaro Obregón   Tláhuac                    1
                 Tultitlán                  1
                 Venustiano Carranza       27
                 Xochimilco                11
                 Zumpango                   1

[1508 rows x 1 columns]

In [78]:
muni_table.reset_index()

,colonia_origen,colonia_destino,id_via
0,Alfredo Baranda,Balbuena,3
1,Alfredo Baranda,Bondojito,2
2,Alfredo Baranda,Buenavista-Reforma,9
3,Alfredo Baranda,CC Interlomas-Lomas de Tecamachalco,2
4,Alfredo Baranda,CC Plaza Neza,1
...,...,...,...
10371,Zumpango,Tultitlán -centro-,2
10372,Zumpango,Viveros,1
10373,Zumpango,Zona Industrial Tabla Honda y San Pablo Xalpa,1
10374,Zumpango,Zonas Industriales Tlalnepantla,3


In [31]:
#Neighbbourhood pivot tables, sorted
nei_table = muni_table = pd.pivot_table(results_new, values='id_via', index=['colonia_origen', 'colonia_destino'], aggfunc=pd.Series.nunique, dropna = True)
nei_table = nei_table.sort_values(by ='id_via', ascending = False )


In [32]:
nei_table

,,id_via
colonia_origen,colonia_destino,
CC Interlomas-Lomas de Tecamachalco,Cuajimalpa,66
Villa Nicolás Romero,Condados de Atizapán-México Nuevo,58
Industrial Sin Nombre-Buenavista 2a Secc,Zonas Industriales Tlalnepantla,56
Coyotepec-Teoloyucan,La Quebrada-Parques Industriales,56
Melchor Ocampo-Joyas de Cuautitlán,La Quebrada-Parques Industriales,53
...,...,...
Parque Cerro de la Estrella,San Andrés Tetepilco,1
Coacalco -cabecera-,U.H. de San Pablo de las Salinas,1
Parque Cerro de la Estrella,Pantitlán,1


In [33]:
#Creating a Municipality - Neighbourhood pivot
mun_col_table = pd.pivot_table(results_new, values='id_via', index=['municipio_origen', 'colonia_origen', 'municipio_destino', 'colonia_destino'], aggfunc=pd.Series.nunique, dropna = True)
mun_col_table = mun_col_table.sort_values(by ='id_via', ascending = False )

In [79]:
#For lack of better visualization, creating an excel file to review
mun_col_table.reset_index()

,municipio_origen,colonia_origen,municipio_destino,colonia_destino,id_via
0,Huixquilucan,CC Interlomas-Lomas de Tecamachalco,Cuajimalpa de Morelos,Cuajimalpa,60
1,Nicolás Romero,Villa Nicolás Romero,Atizapán de Zaragoza,Condados de Atizapán-México Nuevo,58
2,Tultitlán,Industrial Sin Nombre-Buenavista 2a Secc,Tlalnepantla de Baz,Zonas Industriales Tlalnepantla,56
3,Nicolás Romero,Vista Hermosa,Atizapán de Zaragoza,Condados de Atizapán-México Nuevo,48
4,Venustiano Carranza,Moctezuma,Cuauhtémoc,Centro Histórico,45
...,...,...,...,...,...
12210,Ixtapaluca,Pueblos de Ixtapaluca,Iztapalapa,El Molino Tezonco,1
12211,Ixtapaluca,Pueblos de Ixtapaluca,Iztacalco,UPIICSA,1
12212,Ixtapaluca,Pueblos de Ixtapaluca,Iztacalco,Reforma Iztaccihuatl,1
12213,Ixtapaluca,Pueblos de Ixtapaluca,Iztacalco,Pantitlán,1


In [277]:
mun_orig_df = mun_orig_df = pd.pivot_table(results_new, values='id_via', index=['estado_origen', 'municipio_origen'], aggfunc=pd.Series.nunique, dropna = True)
mun_dest_df = mun_dest_df = pd.pivot_table(results_new, values='id_via', index=['estado_destino', 'municipio_destino'], aggfunc=pd.Series.nunique, dropna = True)



In [280]:
mun_dest_df.reset_index()
mun_orig_df.reset_index()

,estado_origen,municipio_origen,id_via
0,Ciudad de México,Azcapotzalco,794
1,Ciudad de México,Benito Juárez,927
2,Ciudad de México,Coyoacán,1396
3,Ciudad de México,Cuajimalpa de Morelos,510
4,Ciudad de México,Cuauhtémoc,915
5,Ciudad de México,Gustavo A. Madero,2474
6,Ciudad de México,Iztacalco,1013
7,Ciudad de México,Iztapalapa,3535
8,Ciudad de México,La Magdalena Contreras,547
9,Ciudad de México,Miguel Hidalgo,608


In [281]:
mun_dest_df.to_csv("mun_dest.csv")
mun_orig_df.to_csv("mun_orig.csv")

In [34]:
mun_col_table

id_via
municipio_origen    colonia_origen                           municipio_destino     colonia_destino                          
Huixquilucan        CC Interlomas-Lomas de Tecamachalco      Cuajimalpa de Morelos Cuajimalpa                             60
Nicolás Romero      Villa Nicolás Romero                     Atizapán de Zaragoza  Condados de Atizapán-México Nuevo      58
Tultitlán           Industrial Sin Nombre-Buenavista 2a Secc Tlalnepantla de Baz   Zonas Industriales Tlalnepantla        56
Nicolás Romero      Vista Hermosa                            Atizapán de Zaragoza  Condados de Atizapán-México Nuevo      48
Venustiano Carranza Moctezuma                                Cuauhtémoc            Centro Histórico                       45
...                                                                                                                      ...
Ixtapaluca          Pueblos de Ixtapaluca                    Iztapalapa            El Molino Tezonco                       1
                                                             Iztacalco             UPIICSA                                 1
                                                                                   Reforma Iztaccihuatl                    1
                                                                                   Pantitlán                               1
Álvaro Obregón      Viveros                                  Venustiano Carranza   Zaragoza                                1

[12215 rows x 1 columns]

In [35]:
mun_col_table.to_excel("mun_col_table.xlsx")

In [ ]:
#Used Excel to un_group municipalities and create an origin-muni-neighbourhood->destination-muni-neighbourhood table


In [36]:
#Reading back from excel the un-merged cell file and fi
m_c_full = pd.read_excel (r'mun_col_unm.xlsx')
m_c_full = m_c_full.fillna(method='ffill')

In [284]:
muni_o

,municipio_origen,id_via
0,Acolman,316
1,Amecameca,23
2,Apaxco,6
3,Atenco,118
4,Atizapán de Zaragoza,793
5,Atlautla,16
6,Axapusco,11
7,Ayapango,4
8,Azcapotzalco,792
9,Benito Juárez,924


In [285]:
#Build API Calls Test - commenting to simplify
#url = f"http://api.openweathermap.org/data/2.5/weather?appid={weather_api_key}&units=metric"

#cit = []
#cou = []
#lat = []
#lng = []


#for municipo_origen in muni_o:
    #print(f"{url}&q={city}")
 #   response = requests.get(f"{url}&q={city}")
    #print(response)
  #  response = response.json()
#    try:
 #       cit.append(response["name"])
  #      cou.append(response["sys"]["country"])
   #     lng.append(response["coord"]["lon"])
    #    lat.append(response["coord"]["lat"])
    #except:
     #   continue

In [72]:
#Defining my lists to append data
orig = []
dest = []
dist = []
time = []
traff = []

params = {
"key": g_key,
"units": "metric",
"origins": "parque de la estrella, iztapalapa",
"destinations": "centro histórico, CDMX,",
"departure_time": "now"
}

#API URL
distance_metric_url = "https://maps.googleapis.com/maps/api/distancematrix/json"

#Request
response = requests.get(distance_metric_url, params = params).json()

dest.append(response['destination_addresses'])
orig.append(response['origin_addresses'])
dist.append(response['rows'][0]['elements'][0]['distance']['value'])
time.append(response['rows'][0]['elements'][0]['duration']['value'])
traff.append(response['rows'][0]['elements'][0]['duration_in_traffic']['value'])

print(response)


{'destination_addresses': ['Historic center of Mexico City, Centro, Mexico City, CDMX, Mexico'], 'origin_addresses': ['Cerro de la Estrella National Park, Carretera Escenica al Cerro de la Estrella, Amp Veracruzana, Iztapalapa, 09856 Ciudad de México, CDMX, Mexico'], 'rows': [{'elements': [{'distance': {'text': '16.0 km', 'value': 15955}, 'duration': {'text': '38 mins', 'value': 2253}, 'duration_in_traffic': {'text': '53 mins', 'value': 3161}, 'status': 'OK'}]}], 'status': 'OK'}


In [77]:
response

{'destination_addresses': ['Historic center of Mexico City, Centro, Mexico City, CDMX, Mexico'],
 'origin_addresses': ['Cerro de la Estrella National Park, Carretera Escenica al Cerro de la Estrella, Amp Veracruzana, Iztapalapa, 09856 Ciudad de México, CDMX, Mexico'],
 'rows': [{'elements': [{'distance': {'text': '16.0 km', 'value': 15955},
     'duration': {'text': '38 mins', 'value': 2253},
     'duration_in_traffic': {'text': '53 mins', 'value': 3161},
     'status': 'OK'}]}],
 'status': 'OK'}

In [82]:
Colonias_Target_API = pd.read_excel (r'Colonias_Target_API.xlsx')

In [127]:
#Defining my lists to append data
orig = []
dest = []
dist = []
timep = []
traff = []
timest = []

params = {
"key": g_key,
"units": "metric",
"departure_time": "now"
}

#API URL
distance_metric_url = "https://maps.googleapis.com/maps/api/distancematrix/json"


for index, row in Colonias_Target_API.iterrows():

    # get info type from df
    start_point = row['Start_travel']
    end_point = row['End_travel']

    # add keyword to params dict
    params['origins'] = start_point
    params['destinations'] = end_point

    # assemble url and make API request
    #Request
    response = requests.get(distance_metric_url, params = params).json()

    dest.append(response['destination_addresses'])
    orig.append(response['origin_addresses'])
    dist.append(response['rows'][0]['elements'][0]['distance']['value'])
    time.append(response['rows'][0]['elements'][0]['duration']['value'])
    traff.append(response['rows'][0]['elements'][0]['duration_in_traffic']['value'])
    timest.append(pd.Series([dt.datetime.now()]))

print(response)
    


{'destination_addresses': ['Portales Nte, Ciudad de México, CDMX, Mexico'], 'origin_addresses': ['Jardines de Morelos, 55070 Ecatepec de Morelos, Méx., Mexico'], 'rows': [{'elements': [{'distance': {'text': '38.6 km', 'value': 38626}, 'duration': {'text': '52 mins', 'value': 3128}, 'duration_in_traffic': {'text': '42 mins', 'value': 2519}, 'status': 'OK'}]}], 'status': 'OK'}


In [95]:
dest

[['Historic center of Mexico City, Centro, Mexico City, CDMX, Mexico'],
 ['Lomas de Chapultepec, 11000 Mexico City, CDMX, Mexico'],
 ['Del Valle, Mexico City, CDMX, Mexico'],
 ['Historic center of Mexico City, Centro, Mexico City, CDMX, Mexico'],
 ['Lomas de Chapultepec, 11000 Mexico City, CDMX, Mexico'],
 ['Colonia Nápoles, Mexico City, CDMX, Mexico'],
 ['Historic center of Mexico City, Centro, Mexico City, CDMX, Mexico'],
 ['Lomas de Chapultepec, 11000 Mexico City, CDMX, Mexico'],
 ['Portales Nte, Ciudad de México, CDMX, Mexico']]

In [96]:
orig

[['Santa Martha Acatitla, Ciudad de México, CDMX, Mexico'],
 ['Av. San Rafael Atlixco 186, Leyes de Reforma 1ra Secc, Iztapalapa, 09340 Ciudad de México, CDMX, Mexico'],
 ['Lomas Estrella, Ciudad de México, CDMX, Mexico'],
 ['Bondojito, Ciudad de México, CDMX, Mexico'],
 ['Bondojito, Ciudad de México, CDMX, Mexico'],
 ['Villa Gustavo A. Madero, Mexico City, CDMX, Mexico'],
 ['Av. Carlos Hank González 50, Valle de Anahuac, 55210 Ecatepec de Morelos, Méx., Mexico'],
 ['Av. Carlos Hank González 50, Valle de Anahuac, 55210 Ecatepec de Morelos, Méx., Mexico'],
 ['Jardines de Morelos, 55070 Ecatepec de Morelos, Méx., Mexico']]

In [97]:
time

[2097, 2390, 2297, 1253, 1824, 1782, 1840, 2435, 3128]

In [98]:
dist

[18424, 22789, 15939, 7520, 16948, 17200, 17937, 29229, 38626]

In [99]:
traff

[1780, 1958, 1647, 926, 1408, 1370, 1513, 2040, 2583]

In [100]:
four_fifty_df = pd.DataFrame()
four_fifty_df['Starting Point Address'] = orig
four_fifty_df['End Point Address'] = dest
four_fifty_df['Route Distance'] = dist
four_fifty_df['Nominal Time'] = time
four_fifty_df['Route Traffic Time'] = traff



In [199]:
Colonias_Target_API

,Unnamed: 0,Colonia_origen,Colonia_destino,Municipio_origen,Municipio_destino,Start_travel,End_travel
0,0,Santa Martha Acatitla,Centro Histórico,Iztapalapa,Cuauhtémoc,"colonia Santa Martha Acatitla, Iztapalapa, Ciu...","colonia Centro Histórico, Cuauhtémoc, CDMX"
1,1,UAM Iztapalapa,Chapultepec-Polanco,Iztapalapa,Miguel Hidalgo,"colonia UAM Iztapalapa, Iztapalapa, Ciudad de ...","colonia Chapultepec-Polanco, Miguel Hidalgo, CDMX"
2,2,Lomas Estrella,Del Valle,Iztapalapa,Benito Juárez,"colonia Lomas Estrella, Iztapalapa, Ciudad de ...","colonia Del Valle, Benito Juárez, CDMX"
3,3,Bondojito,Centro Histórico,Gustavo A. Madero,Cuauhtémoc,"colonia Bondojito, Gustavo A. Madero, Ciudad d...","colonia Centro Histórico, Cuauhtémoc, CDMX"
4,4,Bondojito,Chapultepec-Polanco,Gustavo A. Madero,Miguel Hidalgo,"colonia Bondojito, Gustavo A. Madero, Ciudad d...","colonia Chapultepec-Polanco, Miguel Hidalgo, CDMX"
5,5,La Villa,Napoles,Gustavo A. Madero,Benito Juárez,"colonia La Villa, Gustavo A. Madero, Ciudad de...","colonia Napoles, Benito Juárez, CDMX"
6,6,CC Center Plazas-Nueva Aragón,Centro Histórico,Ecatepec de Morelos,Cuauhtémoc,"colonia CC Center Plazas-Nueva Aragón, Ecatepe...","colonia Centro Histórico, Cuauhtémoc, CDMX"
7,7,CC Center Plazas-Nueva Aragón,Chapultepec-Polanco,Ecatepec de Morelos,Miguel Hidalgo,"colonia CC Center Plazas-Nueva Aragón, Ecatepe...","colonia Chapultepec-Polanco, Miguel Hidalgo, CDMX"
8,8,Jardines de Morelos,Portales,Ecatepec de Morelos,Benito Juárez,"colonia Jardines de Morelos, Ecatepec de Morel...","colonia Portales, Benito Juárez, CDMX"


In [101]:
four_fifty_df

,Starting Point Address,End Point Address,Route Distance,Nominal Time,Route Traffic Time
0,"[Santa Martha Acatitla, Ciudad de México, CDMX...","[Historic center of Mexico City, Centro, Mexic...",18424,2097,1780
1,"[Av. San Rafael Atlixco 186, Leyes de Reforma ...","[Lomas de Chapultepec, 11000 Mexico City, CDMX...",22789,2390,1958
2,"[Lomas Estrella, Ciudad de México, CDMX, Mexico]","[Del Valle, Mexico City, CDMX, Mexico]",15939,2297,1647
3,"[Bondojito, Ciudad de México, CDMX, Mexico]","[Historic center of Mexico City, Centro, Mexic...",7520,1253,926
4,"[Bondojito, Ciudad de México, CDMX, Mexico]","[Lomas de Chapultepec, 11000 Mexico City, CDMX...",16948,1824,1408
5,"[Villa Gustavo A. Madero, Mexico City, CDMX, M...","[Colonia Nápoles, Mexico City, CDMX, Mexico]",17200,1782,1370
6,"[Av. Carlos Hank González 50, Valle de Anahuac...","[Historic center of Mexico City, Centro, Mexic...",17937,1840,1513
7,"[Av. Carlos Hank González 50, Valle de Anahuac...","[Lomas de Chapultepec, 11000 Mexico City, CDMX...",29229,2435,2040
8,"[Jardines de Morelos, 55070 Ecatepec de Morelo...","[Portales Nte, Ciudad de México, CDMX, Mexico]",38626,3128,2583


In [226]:
destn = []
orign = []
distn = []
timen = []
traffn = []
timestn = []

In [201]:
destm

[]

In [245]:
#Final API Calls to get traffic data between the selected routes
for index, row in Colonias_Target_API.iterrows():

    # get restaurant type from df
    start_point = row['Start_travel']
    end_point = row['End_travel']

    # add keyword to params dict
    params['origins'] = start_point
    params['destinations'] = end_point

    #time.sleep(300)
    
    # assemble url and make API request
    #Request
    response = requests.get(distance_metric_url, params = params).json()

    destn.append(response['destination_addresses'])
    orign.append(response['origin_addresses'])
    distn.append(response['rows'][0]['elements'][0]['distance']['value'])
    timen.append(response['rows'][0]['elements'][0]['duration']['value'])
    traffn.append(response['rows'][0]['elements'][0]['duration_in_traffic']['value'])
    timestn.append(pd.Series([dt.datetime.now()]))



print(response)
mars_df = pd.DataFrame()
mars_df['Starting Point Address'] = orign
mars_df['End Point Address'] = destn
mars_df['Route Distance'] = distn
mars_df['Nominal Time'] = timen
mars_df['Route Traffic Time'] = traffn
mars_df['Timestamp'] = timestn
    
#import datetime as dt
mars_df

{'destination_addresses': ['Portales Nte, Ciudad de México, CDMX, Mexico'], 'origin_addresses': ['Jardines de Morelos, 55070 Ecatepec de Morelos, Méx., Mexico'], 'rows': [{'elements': [{'distance': {'text': '48.2 km', 'value': 48201}, 'duration': {'text': '58 mins', 'value': 3487}, 'duration_in_traffic': {'text': '1 hour 8 mins', 'value': 4062}, 'status': 'OK'}]}], 'status': 'OK'}


,Starting Point Address,End Point Address,Route Distance,Nominal Time,Route Traffic Time,Timestamp
0,"[Santa Martha Acatitla, Ciudad de México, CDMX...","[Historic center of Mexico City, Centro, Mexic...",18424,2097,1769,0 2020-11-03 05:07:30.537396 dtype: datetime...
1,"[Av. San Rafael Atlixco 186, Leyes de Reforma ...","[Lomas de Chapultepec, 11000 Mexico City, CDMX...",24444,2440,1797,0 2020-11-03 05:07:31.771059 dtype: datetime...
2,"[Lomas Estrella, Ciudad de México, CDMX, Mexico]","[Del Valle, Mexico City, CDMX, Mexico]",15939,2297,1661,0 2020-11-03 05:07:32.586377 dtype: datetime...
3,"[Bondojito, Ciudad de México, CDMX, Mexico]","[Historic center of Mexico City, Centro, Mexic...",7520,1253,889,0 2020-11-03 05:07:33.428051 dtype: datetime...
4,"[Bondojito, Ciudad de México, CDMX, Mexico]","[Lomas de Chapultepec, 11000 Mexico City, CDMX...",16441,1783,1394,0 2020-11-03 05:07:33.920688 dtype: datetime...
5,"[Villa Gustavo A. Madero, Mexico City, CDMX, M...","[Colonia Nápoles, Mexico City, CDMX, Mexico]",17200,1782,1371,0 2020-11-03 05:07:34.965863 dtype: datetime...
6,"[Av. Carlos Hank González 50, Valle de Anahuac...","[Historic center of Mexico City, Centro, Mexic...",17937,1840,1508,0 2020-11-03 05:07:35.796822 dtype: datetime...
7,"[Av. Carlos Hank González 50, Valle de Anahuac...","[Lomas de Chapultepec, 11000 Mexico City, CDMX...",29229,2435,2050,0 2020-11-03 05:07:37.031153 dtype: datetime...
8,"[Jardines de Morelos, 55070 Ecatepec de Morelo...","[Portales Nte, Ciudad de México, CDMX, Mexico]",38626,3128,2585,0 2020-11-03 05:07:38.324676 dtype: datetime...
9,"[Santa Martha Acatitla, Ciudad de México, CDMX...","[Historic center of Mexico City, Centro, Mexic...",18424,2097,1810,0 2020-11-03 05:10:02.947043 dtype: datetime...


In [246]:
mar_df

,Starting Point Address,End Point Address,Route Distance,Nominal Time,Route Traffic Time,Timestamp
0,"[Santa Martha Acatitla, Ciudad de México, CDMX...","[Historic center of Mexico City, Centro, Mexic...",18424,2097,1808,0 2020-11-02 22:50:46.180789 dtype: datetime...
1,"[Av. San Rafael Atlixco 186, Leyes de Reforma ...","[Lomas de Chapultepec, 11000 Mexico City, CDMX...",23296,2577,2071,0 2020-11-02 22:50:47.725199 dtype: datetime...
2,"[Lomas Estrella, Ciudad de México, CDMX, Mexico]","[Del Valle, Mexico City, CDMX, Mexico]",16010,2304,1753,0 2020-11-02 22:50:49.210927 dtype: datetime...
3,"[Bondojito, Ciudad de México, CDMX, Mexico]","[Historic center of Mexico City, Centro, Mexic...",7520,1253,954,0 2020-11-02 22:50:50.335152 dtype: datetime...
4,"[Bondojito, Ciudad de México, CDMX, Mexico]","[Lomas de Chapultepec, 11000 Mexico City, CDMX...",16456,1813,1425,0 2020-11-02 22:50:51.284132 dtype: datetime...
5,"[Villa Gustavo A. Madero, Mexico City, CDMX, M...","[Colonia Nápoles, Mexico City, CDMX, Mexico]",17200,1782,1482,0 2020-11-02 22:50:52.486436 dtype: datetime...
6,"[Av. Carlos Hank González 50, Valle de Anahuac...","[Historic center of Mexico City, Centro, Mexic...",17937,1840,1573,0 2020-11-02 22:50:53.364642 dtype: datetime...
7,"[Av. Carlos Hank González 50, Valle de Anahuac...","[Lomas de Chapultepec, 11000 Mexico City, CDMX...",29229,2435,2127,0 2020-11-02 22:50:54.331739 dtype: datetime...
8,"[Jardines de Morelos, 55070 Ecatepec de Morelo...","[Portales Nte, Ciudad de México, CDMX, Mexico]",38626,3128,2714,0 2020-11-02 22:50:55.968766 dtype: datetime...
9,"[Santa Martha Acatitla, Ciudad de México, CDMX...","[Historic center of Mexico City, Centro, Mexic...",18424,2097,1808,0 2020-11-02 22:52:38.057236 dtype: datetime...


In [248]:
dfs = [mar_df, mars_df]
final_df_tue = pd.concat(dfs)
final_df_tue

,Starting Point Address,End Point Address,Route Distance,Nominal Time,Route Traffic Time,Timestamp
0,"[Santa Martha Acatitla, Ciudad de México, CDMX...","[Historic center of Mexico City, Centro, Mexic...",18424,2097,1808,0 2020-11-02 22:50:46.180789 dtype: datetime...
1,"[Av. San Rafael Atlixco 186, Leyes de Reforma ...","[Lomas de Chapultepec, 11000 Mexico City, CDMX...",23296,2577,2071,0 2020-11-02 22:50:47.725199 dtype: datetime...
2,"[Lomas Estrella, Ciudad de México, CDMX, Mexico]","[Del Valle, Mexico City, CDMX, Mexico]",16010,2304,1753,0 2020-11-02 22:50:49.210927 dtype: datetime...
3,"[Bondojito, Ciudad de México, CDMX, Mexico]","[Historic center of Mexico City, Centro, Mexic...",7520,1253,954,0 2020-11-02 22:50:50.335152 dtype: datetime...
4,"[Bondojito, Ciudad de México, CDMX, Mexico]","[Lomas de Chapultepec, 11000 Mexico City, CDMX...",16456,1813,1425,0 2020-11-02 22:50:51.284132 dtype: datetime...
...,...,...,...,...,...,...
166,"[Bondojito, Ciudad de México, CDMX, Mexico]","[Lomas de Chapultepec, 11000 Mexico City, CDMX...",14885,2178,2625,0 2020-11-03 08:20:18.740687 dtype: datetime...
167,"[Villa Gustavo A. Madero, Mexico City, CDMX, M...","[Colonia Nápoles, Mexico City, CDMX, Mexico]",16206,2043,2564,0 2020-11-03 08:20:19.591927 dtype: datetime...
168,"[Av. Carlos Hank González 50, Valle de Anahuac...","[Historic center of Mexico City, Centro, Mexic...",17937,1840,2335,0 2020-11-03 08:20:20.308493 dtype: datetime...
169,"[Av. Carlos Hank González 50, Valle de Anahuac...","[Lomas de Chapultepec, 11000 Mexico City, CDMX...",32487,2626,3634,0 2020-11-03 08:20:21.553347 dtype: datetime...


In [249]:
final_df_tue.to_excel("df_timepoints.xlsx")